In [1]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=10,
    length_function=len,
    add_start_index=True,
)
context = ''' CHRIST (Deemed to be University) was born out of the educational
vision of St Kuriakose Elias Chavara, an educationalist and social reformer of
the nineteenth century in South India. He founded the first Catholic
indigenous congregation, Carmelites of Mary Immaculate (CMI), in 1831
which administers CHRIST (Deemed to be University). “CHRIST (Deemed to
be University) was established as ‘Christ College’ in 1969. It undertook path-
breaking initiatives in Indian higher education with the introduction of
innovative and modern curricula, insistence on academic discipline,
imparting of Holistic Education and adoption of global higher education
practices with the support of creative and dedicated staff.” '''

